In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
%matplotlib inline

In [43]:
raw_data = pd.read_csv('IPL_Matches_2008_2022.csv')
raw_data.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [44]:
raw_data = raw_data[['ID', 'City', 'Team1', 'Team2', 'WinningTeam']]

In [45]:
raw_data.shape

(950, 5)

### Many teams not playing now ipl so fixing with new data
    Kings XI Punjab named Punjab Kings  
    Delhi Daredevils named Delhi Capitals  
    Deccan Chargers named Sunrisers Hyderabad  
    Rising Pune Supergiant, Rising Pune Supergiants, Pune Warriors, Gujarat Lions, Kochi Tuskers Kerala dont play matches anymore

In [46]:
def change_names(obj: str):
    not_playing = [
    'Rising Pune Supergiant', 
    'Rising Pune Supergiants', 
    'Pune Warriors', 
    'Gujarat Lions', 
    'Kochi Tuskers Kerala'
    ]
    if obj in not_playing:
        return np.nan
    elif obj == 'Kings XI Punjab':
        return 'Punjab Kings'
    elif obj == 'Delhi Daredevils':
        return 'Delhi Capitals'
    elif obj == 'Deccan Chargers':
        return 'Sunrisers Hyderabad'
    else:
        return obj

In [47]:
raw_data['Team1'] = raw_data['Team1'].apply(change_names)
raw_data['Team2'] = raw_data['Team2'].apply(change_names)
raw_data['WinningTeam'] = raw_data['WinningTeam'].apply(change_names)

In [48]:
raw_data.dropna(axis=0, inplace=True)

In [49]:
raw_data.shape

(781, 5)

In [50]:
raw_data.head()

,ID,City,Team1,Team2,WinningTeam
0,1312200,Ahmedabad,Rajasthan Royals,Gujarat Titans,Gujarat Titans
1,1312199,Ahmedabad,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals
2,1312198,Kolkata,Royal Challengers Bangalore,Lucknow Super Giants,Royal Challengers Bangalore
3,1312197,Kolkata,Rajasthan Royals,Gujarat Titans,Gujarat Titans
4,1304116,Mumbai,Sunrisers Hyderabad,Punjab Kings,Punjab Kings


### Ball wise dataset

In [51]:
raw_data1 = pd.read_csv('IPL_Ball_by_Ball_2008_2022.csv')
raw_data1.head(2)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals


In [53]:
total_run = pd.DataFrame(raw_data1.groupby(['ID', 'innings'], sort=False)['total_run'].sum()).reset_index()
total_run.head(4)

,ID,innings,total_run
0,1312200,1,130
1,1312200,2,133
2,1312199,1,157
3,1312199,2,161


In [55]:
matches = pd.merge(left=raw_data, right=total_run, on='ID')
matches = matches[matches['innings']==1]
matches = matches[['ID', 'City', 'WinningTeam', 'total_run', 'Team1', 'Team2']]
matches.head()

,ID,City,WinningTeam,total_run,Team1,Team2
0,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans
2,1312199,Ahmedabad,Rajasthan Royals,157,Royal Challengers Bangalore,Rajasthan Royals
4,1312198,Kolkata,Royal Challengers Bangalore,207,Royal Challengers Bangalore,Lucknow Super Giants
6,1312197,Kolkata,Gujarat Titans,188,Rajasthan Royals,Gujarat Titans
8,1304116,Mumbai,Punjab Kings,157,Sunrisers Hyderabad,Punjab Kings


In [57]:
deliveries = pd.merge(left=matches, right=raw_data1, on='ID')
deliveries.head()

,ID,City,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [13]:
deliveries['cur_run'] = deliveries.groupby('ID')['total_run_y'].cumsum()

In [14]:
deliveries['runs_left'] = (deliveries['total_run_x'] - deliveries['cur_run']) + 1

In [15]:
deliveries['balls_left'] = 126 - (deliveries['overs']*6 + deliveries['ballnumber'])

In [16]:
deliveries['player_out'].fillna(0, inplace=True)
deliveries['player_out'] = deliveries['player_out'].apply(lambda x:x if x==0 else 1)
deliveries['player_out'] = deliveries['player_out'].astype(int)

In [17]:
deliveries['wicket_left'] = 10 - deliveries.groupby('ID')['player_out'].cumsum()

In [18]:
deliveries['balling_team'] = np.zeros_like(deliveries['BattingTeam'])

In [19]:
for i in range(len(deliveries)):
    if deliveries.iloc[i, 21] == deliveries.iloc[i, 4]:
        deliveries.iloc[i, 26] = deliveries.iloc[i, 5]
    else:
        deliveries.iloc[i, 26] = deliveries.iloc[i, 4]

In [20]:
deliveries['crr'] = np.round((deliveries['cur_run']*6)/(126-deliveries['balls_left']), 2)

In [21]:
deliveries['rrr'] = np.round((deliveries['runs_left']*6)/deliveries['balls_left'], 2)

In [22]:
def win_or_not(df):
    if df['WinningTeam'] == df['BattingTeam']:
        return 1
    else:
        return 0

In [23]:
deliveries['win_or_loss'] = deliveries.apply(win_or_not, axis=1)

In [24]:
deliveries.head(5)

,ID,City,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,cur_run,runs_left,balls_left,wicket_left,balling_team,crr,rrr,win_or_loss
0,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,0,NaN,NaN,Rajasthan Royals,0,131,125,10,Gujarat Titans,0.0,6.29,0
1,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,0,NaN,NaN,Rajasthan Royals,1,130,124,10,Gujarat Titans,3.0,6.29,0
2,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,0,NaN,NaN,Rajasthan Royals,2,129,123,10,Gujarat Titans,4.0,6.29,0
3,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,0,NaN,NaN,Rajasthan Royals,2,129,122,10,Gujarat Titans,3.0,6.34,0
4,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,0,NaN,NaN,Rajasthan Royals,2,129,121,10,Gujarat Titans,2.4,6.40,0


In [25]:
def change(obj):
    if obj == 'Kings XI Punjab':
        return 'Punjab Kings'
    elif obj == 'Delhi Daredevils':
        return 'Delhi Capitals'
    elif obj == 'Deccan Chargers':
        return 'Sunrisers Hyderabad'
    else:
        return obj

In [26]:
deliveries['BattingTeam'] = deliveries['BattingTeam'].apply(change)

In [27]:
final_data = deliveries[[
    'BattingTeam', 'balling_team', 'City', 'runs_left', 'balls_left', 'wicket_left', 'total_run_x', 'crr', 'rrr', 'win_or_loss'
]]
final_data.columns = [
    'Batting', 'Bowling', 'City', 'Runs_left', 'Balls_left', 'Wicket_left', 'Total_run', 'crr', 'rrr', 'Result'
]
final_data.head(5)

,Batting,Bowling,City,Runs_left,Balls_left,Wicket_left,Total_run,crr,rrr,Result
0,Rajasthan Royals,Gujarat Titans,Ahmedabad,131,125,10,130,0.0,6.29,0
1,Rajasthan Royals,Gujarat Titans,Ahmedabad,130,124,10,130,3.0,6.29,0
2,Rajasthan Royals,Gujarat Titans,Ahmedabad,129,123,10,130,4.0,6.29,0
3,Rajasthan Royals,Gujarat Titans,Ahmedabad,129,122,10,130,3.0,6.34,0
4,Rajasthan Royals,Gujarat Titans,Ahmedabad,129,121,10,130,2.4,6.40,0


In [28]:
final_data.shape

(186229, 10)

In [29]:
final_data.isna().sum()

Batting        0
Bowling        0
City           0
Runs_left      0
Balls_left     0
Wicket_left    0
Total_run      0
crr            0
rrr            0
Result         0
dtype: int64

In [30]:
final_data.to_csv('data.csv', index=False, header=True)

# Insert into mysql

In [31]:
import mysql.connector
  
db = mysql.connector.connect(
  host ="localhost",
  user ="root",
  passwd ="12345678rk"
)
cursor = db.cursor()

In [32]:
cursor.execute('show databases')
cursor.fetchall()

[('information_schema',), ('mysql',), ('performance_schema',), ('test',)]

In [33]:
cursor.execute('create database ipl')
db.commit()

In [34]:
cursor.execute('show databases')
cursor.fetchall()

[('information_schema',),
 ('ipl',),
 ('mysql',),
 ('performance_schema',),
 ('test',)]

In [35]:
from sqlalchemy import create_engine
engine = create_engine(
    "mysql+pymysql://" + "root" + ":" + "12345678rk" + "@" + "localhost" + ":" + "3306" + "/" + "ipl" + "?" + "charset=utf8mb4"
)

In [36]:
final_data.to_sql('model_data', engine, index=False)

186229